In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/engineering-placements-prediction/collegePlace.csv')

df.head()

In [ ]:
df['Age'].value_counts()

In [ ]:
df.columns.isnull().sum()

In [ ]:
df['Stream'].value_counts()

In [ ]:
df['PlacedOrNot'].value_counts()

In [ ]:
cat_cols =['Gender','Stream']

from sklearn.preprocessing import LabelEncoder

lb=LabelEncoder()

df['Gender']=lb.fit_transform(df.Gender.values)
df['Stream']=lb.fit_transform(df.Stream.values)


df.head()


In [ ]:
df.corr()

In [ ]:
from xgboost import XGBClassifier

X=df.iloc[:, :-1]
y=df['PlacedOrNot']

model = XGBClassifier()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
model.fit(X_train,y_train)

y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred,y_test)

In [ ]:
from sklearn.metrics import classification_report

print (classification_report(y_pred,y_test))

In [ ]:
from numpy import std,mean
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:


def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    
    model = XGBClassifier(
        random_state=42,
        tree_method="gpu_hist",
        gpu_id=1,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(X_train, y_train, early_stopping_rounds=300, eval_set=[(X_test, y_test)], verbose=1000)
    y_preds = model.predict(X_test)
    acc=accuracy_score(y_preds,y_test)
    return acc

In [ ]:
import optuna

study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=5)

In [ ]:
output = pd.DataFrame({'Id': X_test.index,
                       'PlacedOrNot': y_pred})
output.to_csv('/kaggle/working/submission.csv', index=False)
